In [2]:
import os
import numpy as np
import scipy.io
import pandas as pd

In [16]:
dblp_data_fold = '../data/dblp/'
pa_data = pd.read_csv(os.path.join(dblp_data_fold,'paper_author.txt'),sep='\t',names={'PA_P','PA_A'})
pc_data = pd.read_csv(os.path.join(dblp_data_fold,'paper_conf.txt'),sep='\t',names=['PC_P','PC_C'])
n_p = len(set(pa_data['PA_P']))
n_a = len(set(pa_data['PA_A']))
n_c = len(set(pc_data['PC_C']))
n_p, n_a, n_c

(14376, 14475, 20)

In [18]:
map_pa_data = pa_data.copy()
map_pc_data = pc_data.copy()
map_pa_data.iloc[:, 1] = map_pa_data.iloc[:, 1].apply(lambda x: x + n_p)
map_pc_data.iloc[:, 1] = map_pc_data.iloc[:, 1].apply(lambda x: x + n_p + n_a)

In [25]:
min(map_pa_data['PA_P'])

0

In [40]:
from scipy.sparse import csr_matrix
row = np.array(pa_data['PA_P']).tolist()
col = np.array(pa_data['PA_A']).tolist()
data = np.ones(len(row))
pa_csr_mtx = csr_matrix((data, (row, col)), shape=(n_p, n_a))

In [47]:
row = np.array(pc_data['PC_P']).tolist()
col = np.array(pc_data['PC_C']).tolist()
data = np.ones(len(row))
pc_csr_mtx = csr_matrix((data, (row, col)), shape=(n_p, n_c))

In [48]:
pc_csr_mtx, pa_csr_mtx.transpose()

(<14376x20 sparse matrix of type '<type 'numpy.float64'>'
 	with 14376 stored elements in Compressed Sparse Row format>,
 <14475x14376 sparse matrix of type '<type 'numpy.float64'>'
 	with 41794 stored elements in Compressed Sparse Column format>)

In [51]:
apc_csr_mtx = pa_csr_mtx.transpose() * pc_csr_mtx

In [52]:
apcpa_csr_mtx = apc_csr_mtx * apc_csr_mtx.transpose()

In [53]:
pcp_csr_mtx = pc_csr_mtx * pc_csr_mtx.transpose()
cpa_csr_mtx = pc_csr_mtx.transpose() * pa_csr_mtx
cpapc_csr_mtx = cpa_csr_mtx * cpa_csr_mtx.transpose()

In [72]:
np.nonzero(pa_csr_mtx)

(array([    0,     0,     1, ..., 14375, 14375, 14375], dtype=int32),
 array([7383, 8503, 2492, ..., 3346, 3822, 9024], dtype=int32))

In [98]:
# metapath matrix

row = np.concatenate((np.nonzero(apcpa_csr_mtx)[0],np.nonzero(pa_csr_mtx.transpose())[0],np.nonzero(apc_csr_mtx)[0],
               np.nonzero(pa_csr_mtx)[0]+n_a,np.nonzero(pcp_csr_mtx)[0]+n_a,np.nonzero(pc_csr_mtx)[0]+n_a,
               np.nonzero(cpa_csr_mtx)[0]+n_a+n_p,np.nonzero(pc_csr_mtx.transpose())[0]+n_a+n_p,np.nonzero(cpapc_csr_mtx)[0]+n_a+n_p))

col = np.concatenate((np.nonzero(apcpa_csr_mtx)[1],np.nonzero(pa_csr_mtx.transpose())[1],np.nonzero(apc_csr_mtx)[1]+n_a+n_p,
               np.nonzero(pa_csr_mtx)[1],np.nonzero(pcp_csr_mtx)[1]+n_a,np.nonzero(pc_csr_mtx)[1]+n_a+n_p,
               np.nonzero(cpa_csr_mtx)[1],np.nonzero(pc_csr_mtx.transpose())[1]+n_a,np.nonzero(cpapc_csr_mtx)[1]+n_a+n_p))

In [100]:
max(row), max(col), n_a+n_p

(28870, 28870, 28851)

In [101]:
data = np.concatenate((apcpa_csr_mtx.data,pa_csr_mtx.transpose().data,apc_csr_mtx.data,
                pa_csr_mtx.data,pcp_csr_mtx.data,pc_csr_mtx.data,
                cpa_csr_mtx.data,pc_csr_mtx.transpose().data,cpapc_csr_mtx.data))
metapath_csr_mtx = csr_matrix((data, (row, col)), shape=(n_a+n_p+n_c, n_a+n_p+n_c))

In [106]:
metapath_csr_mtx.todense()

matrix([[  1.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   1.00000000e+00,   1.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   1.00000000e+00,   1.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   1.00000000e+00],
        ..., 
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
           1.37400000e+03,   8.10000000e+01,   9.61800000e+03],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
           8.89000000e+02,   8.98000000e+02,   5.94100000e+03],
        [  0.00000000e+00,   0.00000000e+00,   1.00000000e+00, ...,
           7.80000000e+01,   5.40000000e+01,   5.90000000e+01]])

In [110]:
import scipy.io
scipy.io.savemat(os.path.join(dblp_data_fold,'metapath_mtx'),{"metapath_mtx":metapath_csr_mtx})